# DALEX: Descriptive mAchine Learning EXplanations

Machine Learning (ML) models are widely used and have various applications in classification or regression. Models created with boosting, bagging, stacking or similar techniques are often used due to their high performance, but such black-box models usually lack of interpretability. `DALEX` package contains various explainers that help to understand the link between input variables and model output. The `single_variable()` explainer extracts conditional response of a model as a function of a single selected variable. It is a wrapper over packages `pdp` and `ALEPlot`. The `single_prediction()` explainer attributes arts of model prediction to articular variables used in the model. It is a wrapper over `breakDown` package. The `variable_dropout()` explainer assess variable importance based on consecutive permutations. All these explainers can be plotted with generic `plot()` function and compared across different models.

This notebook shows how to use the `DALEX` package to explain machine learning models.

If you want to find more details and methodology, see: [A gentle introduction to DALEX with examples](https://pbiecek.github.io/DALEX_docs/).

### Use case: Regression. Apartment prices in Warsaw
To illustrate applications of DALEX to regression problems we will use an artificial dataset apartments available in the DALEX package. Our goal is to predict the price per square meter of an apartment based on selected features such as construction year, surface, floor, number of rooms, district. It should be noted that four of these variables are continuous while the fifth one is a categorical one. Prices are given in Euro.

In [ ]:
library(DALEX)
head(apartments)

The first model is based on linear regression. It will be a simple model without any feature engineering.

In [ ]:
apartments_lm_model <- lm(m2.price ~ construction.year + surface + floor +
                         no.rooms + district, data = apartments)
summary(apartments_lm_model)

The second model is based on the random forest. It’s a very elastic out-of-the-box model.

In [ ]:
library("randomForest")
set.seed(59)

apartments_rf_model <- randomForest(m2.price ~ construction.year + surface + floor +
                      no.rooms + district, data = apartments)
apartments_rf_model

We have also another apartmentsTest dataset that can be used for validation of the model. Below you may see the mean square error on the basis of validation data calculated for both models

In [ ]:
predicted_mi2_lm <- predict(apartments_lm_model, apartmentsTest)
sqrt(mean((predicted_mi2_lm - apartmentsTest$m2.price)^2))

In [ ]:
predicted_mi2_rf <- predict(apartments_rf_model, apartmentsTest)
sqrt(mean((predicted_mi2_rf - apartmentsTest$m2.price)^2))

These two models have very similar performance! Which one should be used?

## Model understanding
Now we introduce three groups of explainers that can be used to boost our understanding of black-box models.

First we need to prepare wrappers for these models. They are in `explainer_lm` and `explainer_rf` objects.

In [ ]:
explainer_lm <- explain(apartments_lm_model,
                       data = apartmentsTest[,2:6], y = apartmentsTest$m2.price)

explainer_rf <- explain(apartments_rf_model,
                       data = apartmentsTest[,2:6], y = apartmentsTest$m2.price)

### Model performance
Function `model_performance()` calculates predictions and residuals for validation dataset apartmentsTest.

Generic `function print()` returns quantiles for residuals.

In [ ]:
mp_lm <- model_performance(explainer_lm)
mp_lm

In [ ]:
mp_rf <- model_performance(explainer_rf)
mp_rf

The generic `plot()` function shows reversed empirical cumulative distribution function for absolute values from residuals. This function presents a fraction of residuals larger than x. 

In [ ]:
plot(mp_lm, mp_rf)

The plot above shows that majority of residuals for the random forest is smaller than residuals for the linear model, yet the small fraction of very large residuals affects the root mean square.

Use the `geom = "boxplot"` parameter for the generic `plot()` function to get an alternative comparison of residuals. The red dot stands for the root mean square.

In [ ]:
plot(mp_lm, mp_rf, geom = "boxplot")

### Feature importance
Model agnostic variable importance is calculated by means of permutations. We simply substract the loss function calculated for validation dataset with permuted values for a single variable from the loss function calculated for validation dataset. 

This method is implemented in the `variable_importance()` function.

In [ ]:
vi_rf <- variable_importance(explainer_rf, loss_function = loss_root_mean_square)
vi_rf

In [ ]:
vi_lm <- variable_importance(explainer_lm, loss_function = loss_root_mean_square)
vi_lm

It is much easier to compare both models when these values are plotted close to each other. The generic `plot()` function may handle both models.

In [ ]:
plot(vi_lm, vi_rf)

### Variable response
Explainers presented in this section are designed to better understand the relation between a variable and a model output.

The variable `construction.year` is interesting as it is important for the random forest model `apartments_rf_model` but not for the linear model `apartments_lm_model`. Let’s see the relation between the variable and the model output.

#### Partial Dependence Plot
We can use PDP plots to compare two or more models. Below we plot PDP for the linear model against the random forest model.

Below we use `variable_response()` from `DALEX`, which calls `pdp::partial` function to calculate PDP response.

In [ ]:
sv_rf  <- single_variable(explainer_rf, variable =  "construction.year", type = "pdp")
sv_lm  <- single_variable(explainer_lm, variable =  "construction.year", type = "pdp")
plot(sv_rf, sv_lm)

It looks like the random forest captures the non-linear relation that cannot be captured by linear models.

#### Accumulated Local Effects Plot
We can also use `single_variable()` function to call `ALEPlot::ALEPlot()` function to calculate the ALE curve for the variable `construction.year`.

In [ ]:
sva_rf  <- single_variable(explainer_rf, variable = "construction.year", type = "ale")
sva_lm  <- single_variable(explainer_lm, variable = "construction.year", type = "ale")
plot(sva_rf, sva_lm)

Results for PDP and ALEP are very similar except that effects for ALEP are centered around 0.

#### Merging Path Plot
An interesting tool that helps to understand what happens with factor variables is a Merging Path Plot for a factor variable district.

In [ ]:
svd_rf  <- single_variable(explainer_rf, variable = "district", type = "factor")
svd_lm  <- single_variable(explainer_lm, variable = "district", type = "factor")

plot(svd_rf, svd_lm)

The three clusters are: the city center (Srodmiescie), districts well communicated with city center (Ochota, Mokotow, Zoliborz) and other districts closer to city boundaries.

Factor variables are handled very differently by random forest and linear model, yet despite these differences both models result in very similar plots.

### Prediction understanding

#### Outlier detection
Function `model_performance()` may be used to identify outliers. 

As you may remember, residuals for random forest were smaller in general, except for a small fraction of very high residuals.

Let’s use the `model_performance()` function to extract and plot residuals against the observed true values.

In [ ]:
mp_rf <- model_performance(explainer_rf)

library(ggplot2)
ggplot(mp_rf, aes(observed, diff)) + geom_point() +
        xlab("Observed") + ylab("Predicted - Observed") +
        ggtitle("Diagnostic plot for the random forest model") + theme_mi2()

Lets see which variables stand behind the model prediction for an apartment with largest residual.

In [ ]:
which.min(mp_rf$diff)

Observation with the largest residual in the random forest model.

In [ ]:
new_apartment <- apartmentsTest[which.min(mp_rf$diff), ]
new_apartment

Function `single_prediction()` generates variable attributions for selected prediction. The generic `plot()` function shows these attributions.

In [ ]:
new_apartment_rf <- single_prediction(explainer_rf, observation = new_apartment)
plot(new_apartment_rf)

Plots confirm that all variables (district, surface, floor, no.rooms) have positive effects as expected. Still, these effects are too small while the final prediction `3505 + 1881` is much smaller than the real price of a square meter `6679`. Let’s see how the linear model behaves for this observation.

In [ ]:
new_apartment_lm <- single_prediction(explainer_lm, observation = new_apartment)
plot(new_apartment_lm, new_apartment_rf)

Prediction for linear model is much closer to the real price of square meter for this apartment.